In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import tree
from sklearn import preprocessing

titanic_dataset = 'docs/titanic2.csv'
breast_cancer_dataset = 'docs/breast_cancer_dataset.csv'

titanic = pd.read_csv(titanic_dataset, header=None, delimiter=',', names=['Pclass', 'sex', 'age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Initial', 'Age_band', 
'Family_size', 'Alone', 'Fare_cat', 'Deck', 'Title', 'Is_Married', 'Survived']) # se lee el csv y se indican el nombre de las columnas

atributos_titanic = titanic.loc[:, 'Pclass':'Is_Married'] # selección de las columnas de atributos
objetivo_titanic = titanic['Survived'] # selección de la columna objetivo

In [2]:
def metodo_evaluacion_robusta(dataset, atributos, objetivo, N_EXP, CV):

    codificador_atributos = preprocessing.OrdinalEncoder() # Codificador adecuado para los atributos
    codificador_atributos.fit(atributos) 
    atributos_codificados = codificador_atributos.transform(atributos)
    #print('Atributos codificados: ',atributos_codificados)
    #print('Tamaño x: ',atributos_codificados.shape)

    codificador_objetivo = preprocessing.LabelEncoder() # Codificador adecuado para el objetivo
    objetivo_codificado = codificador_objetivo.fit_transform(objetivo)
    #print('Tamaño y: ',objetivo_codificado.shape)
    
    #print(codificador_objetivo.classes_)  # Clases detectadas por el codificador para la variable objetivo
    #print('Objetivo codificado: ',objetivo_codificado)
    #print(codificador_objetivo.inverse_transform([0, 1])) #Ordena alfabéticamente

    #print(dataset.shape[0])  # Cantidad total de ejemplos
    #print(pd.Series(objetivo).value_counts(normalize=True))  # Frecuencia total de cada clase de aceptabilidad

    # Dividimos en conjuntos de entrenamiento y prueba los atributos y el objetivo codificado
    atributos_entrenamiento, atributos_prueba, objetivo_entrenamiento, objetivo_prueba = model_selection.train_test_split(
    atributos_codificados, objetivo_codificado,  # Conjuntos de datos a dividir, usando los mismos índices para ambos
    random_state=12345,  # Valor de la semilla aleatoria, para que el muestreo sea reproducible, a pesar de ser aleatorio
    test_size=.20  # Tamaño del conjunto de prueba
    )  

    clasif_arbol_decision = tree.DecisionTreeClassifier() # creamos el clasificador
    clasif_arbol_decision.fit(X=atributos_entrenamiento, y=objetivo_entrenamiento)

    for i in range(N_EXP):
        print('Iteración: ',i)
        scores = model_selection.cross_val_score(estimator=clasif_arbol_decision, X=atributos_prueba, y=objetivo_prueba, cv=CV, scoring='balanced_accuracy')
        print('Score: ',scores)
        promedio = scores.mean()
        print('Promedio: ',scores.mean())
        
    return promedio
    

In [10]:
metodo_evaluacion_robusta(titanic, atributos_titanic, objetivo_titanic, 10, 10)

Iteración:  0
Score:  [0.74025974 0.72077922 0.67532468 0.67532468 0.6038961  0.7625
 0.8        0.5625     0.6375     0.75714286]
Promedio:  0.6935227272727273
Iteración:  1
Score:  [0.69480519 0.83766234 0.67532468 0.72077922 0.6038961  0.7
 0.9        0.5625     0.6375     0.70714286]
Promedio:  0.7039610389610389
Iteración:  2
Score:  [0.81168831 0.72077922 0.67532468 0.67532468 0.64935065 0.7
 0.85       0.5125     0.6375     0.70714286]
Promedio:  0.6939610389610389
Iteración:  3
Score:  [0.76623377 0.83766234 0.67532468 0.72077922 0.57792208 0.775
 0.85       0.6125     0.6375     0.80714286]
Promedio:  0.7260064935064935
Iteración:  4
Score:  [0.85714286 0.69480519 0.67532468 0.72077922 0.6038961  0.65
 0.85       0.6125     0.7        0.80714286]
Promedio:  0.7171590909090908
Iteración:  5
Score:  [0.88311688 0.83766234 0.67532468 0.72077922 0.57792208 0.7
 0.8        0.6125     0.6375     0.75714286]
Promedio:  0.7201948051948053
Iteración:  6
Score:  [0.85714286 0.79220779 0

0.7123538961038961

In [3]:
def algoritmo_sfs(dataset, D):
    solucion_actual = []
    solucion = []
    k=0
    variables_predictoras = dataset.columns.tolist()
    
    nombre_objetivo = variables_predictoras.pop(len(variables_predictoras)-1)
    objetivo = dataset[nombre_objetivo]
    variables_sin_añadir = variables_predictoras
    i=1
    while k < D:
        #variables_sin_añadir = list(set(variables_predictoras) - set(solucion_actual))
        print('Variables sin añadir: ', variables_sin_añadir)
        lista_scores = []
        lista_sol = []
        for v in variables_sin_añadir:
            lista_sol = solucion_actual
            lista_sol.append(dataset[v])
            #solucion_actual.append(dataset[v])
            #print('Solucion actual: ', solucion_actual)
            solucion_temporal = lista_sol
            solucion_temporal = np.reshape(np.ravel(lista_sol), (891,i+k))
            #print(solucion_temporal)
            score = metodo_evaluacion_robusta(dataset, solucion_temporal, objetivo, 1, 3)
            lista_scores.append(score)
            i=i+1
        print('Lista de scores: ',lista_scores)
        mejor_promedio = np.amax(lista_scores)
        mejor_solucion_temporal = variables_sin_añadir[lista_scores.index(mejor_promedio)]
        print('Mejor solucion temporal: ', mejor_solucion_temporal)
        solucion_actual.append(dataset[mejor_solucion_temporal])
        solucion.append(mejor_solucion_temporal)
        print('Solucion: ', solucion)
        #solucion_actual.append(variables_sin_añadir[lista_scores.index(mejor_solucion_temporal)])
        variables_sin_añadir.remove(variables_sin_añadir[lista_scores.index(mejor_promedio)])
        #print('Solucion actual: ',solucion_actual)
        k=k+1

        print("\nTabla respuesta:\n")
        for j in range (0,D):
            frame_data={'solution':solucion, 'score':mejor_promedio}
        df=pd.DataFrame(frame_data)

    return df.sort_values(by=['score'],ascending = False)

In [4]:
algoritmo_sfs(titanic, 5)

Variables sin añadir:  ['Pclass', 'sex', 'age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Initial', 'Age_band', 'Family_size', 'Alone', 'Fare_cat', 'Deck', 'Title', 'Is_Married']
Iteración:  0
Score:  [0.50857143 0.64       0.69285714]
Promedio:  0.6138095238095238
Iteración:  0
Score:  [0.52285714 0.52571429 0.5       ]
Promedio:  0.5161904761904762
Iteración:  0
Score:  [0.46285714 0.47428571 0.51488095]
Promedio:  0.48400793650793644
Iteración:  0
Score:  [0.46857143 0.47428571 0.45892857]
Promedio:  0.4672619047619047
Iteración:  0
Score:  [0.56571429 0.55428571 0.47559524]
Promedio:  0.5318650793650794
Iteración:  0
Score:  [0.53714286 0.51714286 0.57738095]
Promedio:  0.5438888888888889
Iteración:  0
Score:  [0.60571429 0.64571429 0.5422619 ]
Promedio:  0.5978968253968254
Iteración:  0
Score:  [0.55428571 0.47714286 0.48630952]
Promedio:  0.5059126984126984
Iteración:  0
Score:  [0.42       0.56       0.50833333]
Promedio:  0.4961111111111111
Iteración:  0
Score:  [0.61428571 0.5514

,solution,score
0,Pclass,0.562897
1,Family_size,0.562897
2,SibSp,0.562897
3,Fare_cat,0.562897
4,Age_band,0.562897
